In [ ]:
def parse_filename(fname):
    pattern = ".*__(\w+).csv"
    m = re.match(pattern, fname)
    return m.group(1)

def read_csv(fname,dropcols=None, dropzero = None):
    dropcols = dropcols if dropcols else []
    label = parse_filename(fname)
    df = pd.read_csv('./{}'.format(fname),
                    index_col=0, engine = 'openpyxl')
    df = df[[c for c in df.columns if c.endswith('mass_g') and c not in dropcols]].fillna(0.0)
    df.columns = df.columns.str.replace('_mass_g','')
    if dropzero != None:
        df = df[abs(df).T.sum() > 0].reset_index(drop=True)
        df[df < 0] = 0
        df['label'] = label
        df = df.set_index('label')
    return df, label

def pca_analysis(df, label):
    pca = PCA()
    X = pca.fit_transform(df)
    x1 = X[:, 0]
    x2 = X[:, 1]
    
    fig, ax = plt.subplots()
    ax.plot(x1, x2, '.')
    ax.set(
        xlabel="First component",
        ylabel="Second component",
        title="%s: projected onto first two components" % label,
    )
    
    components = pd.DataFrame(pca.components_.T, index=df.columns)
    print_until = 0.9999
    cumulative_explained_variance = 0
    i = 0
    while cumulative_explained_variance < print_until:
        explained = pca.explained_variance_ratio_[i]
        print("=> COMPONENT %d explains %0.2f%% of the variance" % (i, 100*explained))
        top_components = components[i].sort_values(ascending=False)
        top_components = top_components[abs(top_components) > 0.0]
        print(top_components)
        i += 1
        cumulative_explained_variance += explained
    return components

# TODO
def nmf_analysis(df, label):
    nmf = NMF(components=2)
    X = nmf.fit_transform(df)
    x1 = X[:, 0]
    x2 = X[:, 1]
    fig, ax = plt.subplots()
    ax.plot(x1, x2, '.')
    ax.set(
        xlabel="First component",
        ylabel="Second component",
        title="%s: projected onto first two components" % label,
    )
    error = nmf.reconstruction_err_ / float(len(df))
    components = pd.DataFrame(nmf.components_.T, index=df.columns)

def load_and_analyze(filenames, dropcols=None):
    for filename in filenames:
        df, label = read_csv(filename, dropcols)
        print("== %s ==" % label)
        components = pca_analysis(df, label)
        print()
        
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred, classes)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

def evaluate_model(clf, df, labels):
    model_name = '->'.join(next(zip(*clf.steps)))
    print("Model: %s" % model_name)
    scores = cross_val_score(clf, df, labels, cv=3) #cv=TimeSeriesSplit(3))
    mean_score = scores.mean()
    std_score = scores.std()
    print("Accuracy: [%0.4f, %0.4f] (%0.4f +/- %0.4f)"
          % (mean_score - std_score, mean_score + std_score, mean_score, std_score))
    y_pred = cross_val_predict(clf, df, labels, cv=3)
    y_prob = cross_val_predict(clf, df, labels, cv=3, method='predict_proba')
    plot_confusion_matrix(labels, y_pred, labels.unique())
    return y_pred, y_prob, labels, df

def evaluate_modelrfe(clf, df, labels):
    model_name = '->'.join(next(zip(*clf.steps)))
    print("Model: %s" % model_name)
    scores = cross_val_score(clf, df, labels, cv=3)
    mean_score = scores.mean()
    std_score = scores.std()
    print("Accuracy: [%0.4f, %0.4f] (%0.4f +/- %0.4f)"
          % (mean_score - std_score, mean_score + std_score, mean_score, std_score))
    y_pred = cross_val_predict(clf, df, labels, cv=3)
    y_prob = cross_val_predict(clf, df, labels, cv=3, method='predict')
    return y_pred, y_prob, labels, df

#Type: Data Frame. From a selected isotope and its particle events, select all other isotopes associated and drop others not associated with it
def isotope_particle(data, isotope):
    obs = data[data[isotope] > 0.0]
    return obs

#Merging particle splits based on analyte
def merge_particles(df,dfc):
    index_selector = []
    for i in dfc.columns:
        index_selector1 = dfc[dfc[i].notnull()][i].cumsum()
        index_selector.append(index_selector1)
    dfcorrected = df.groupby(index_selector[13]).agg('sum').reset_index(drop=True).drop(columns = 'ID')
    dfcorrected = dfcorrected.replace(0, np.nan)
    return dfcorrected

#displays R-Sqr value for each dissolved calibration
def r_sqr_analyzer(data, R_value):
    c = data[data['r_sqr_counts'] < R_value]
    return c


#user function: loads a list of variable path names and assigns a label 
def load_and_label(pathname, newlabel):
    data1 = pd.DataFrame()
    for i in glob.glob(pathname):
        data, soil_label = read_csv(i,DROPCOLS)
        data1 = pd.concat([data,data1], axis = 0, sort=False)
    data1['newlabel'] = newlabel
    data1 = data1.set_index('newlabel')
    return data1

#bootstrapping method
# parsing DF for test dataset
def holdoutdata(df, perc):
    #making hold out data
    dfnat = df[df.index == 'Natural']
    dfeng = df[df.index == 'Engineered']

    msk = np.random.rand(len(dfnat)) < perc
    mske = np.random.rand(len(dfeng)) < perc
    holdoutdatanat = dfnat[~msk]
    holdoutdataeng = dfeng[~mske]
    trainingdatanat = dfnat[msk]
    trainingdataeng = dfeng[mske]

    #combining holdout data together and labels
    holdoutdata = pd.concat([holdoutdatanat, holdoutdataeng], axis=0)
    holdoutlabels = holdoutdata.index.get_level_values(level='newlabel')
    return holdoutdata, holdoutlabels, trainingdatanat, trainingdataeng

#bootstrapping process. Holdout some data, predict on it and do that N amount of times. Repeated this procedure M amount of times 
def bootstrap(df, N, m):
    #repeat this process M amount of times
    for _ in range(m):
        #holdout 20% of data. 
        holdoutdata, holdoutlabels, trainingdatanat, trainingeng = holdoutdata(df, 0.2)
        #create a NP array from 0.05 to 1.05 with 0.05 step increment
        p = np.arange(0.05, 1.05, 0.05)
        #create an empty dataframe
        df1 = pd.DataFrame()
        #for each 0.05 step from p
        for i in p:
            #create an empty list
            percent = []
            #repeat this N amount of times
            for _ in range(N):
                #sample an i fraction of the training data
                snat = trainingdatanat.sample(frac = i, replace = True)
                seng = trainingdataeng.sample(frac = i, replace = True)
                
                #combine training data and get traininglabels

                trainingdata = pd.concat([snat, seng], axis=0)
                traininglabels = trainingdata.index.get_level_values(level='newlabel')
                
                #create the logistic Regression model with 5-fold cross validation with NMF of 10 componenets
            
                clf = make_pipeline(
                    StandardScaler(with_mean=False),
                    NMF(n_components=10),
                    LogisticRegressionCV(cv=5, max_iter=300)
                )
                #fit with fraction of training set
                clf = clf.fit(trainingdata, traininglabels)
                #score with the heldout test set
                y_score = clf.score(holdoutdata, holdoutlabels)
                #append the score to percent
                percent.append(y_score)
                #put it in a dataframe format
                pcdf = pd.DataFrame(percent)
            df1 = pd.concat([df1, pcdf], axis = 1)
        
        df1.columns = np.arange(0.05, 1.05, 0.05)
        
        average = []
        stdev = []
        for i in df1.columns:
            average.append(df1[i].mean())
            stdev.append(df1[i].std())
        plt.errorbar(np.arange(0.05, 1.05, 0.05), average, yerr = stdev)
        plt.xlabel('% of training data')
        plt.ylabel('Accuracy of model (%)')

#return df_with_prob
def just_data(data):
    new = data.drop(columns = ['newlabel','Engineered','Natural','prob_obs_group'])
    return new

#marginal_probabilities returns out of the total particle events, what is the probability of an individual isotope occurring
def marginal_probabilities(data):
    #return data.count().sort_values(ascending=False) / total_particles(data).sum()
    return (abs(data) > 0.0).mean().sort_values(ascending=False)

#Counts total number of peaks for each isotope
def marginal_particle(data):
    return data[data > 0.0].count().sort_values(ascending=False)

def non_zero_data(data):
    non_zero_rows = data.abs().sum(axis=1) > 0.0
    non_zero_data = data[non_zero_rows]
    non_zero_columns = non_zero_data.abs().sum(axis=0) > 0.0
    non_zero_data = non_zero_data.loc[: , non_zero_columns]
    return non_zero_data

#counts the total particles associated to an isotope
#conditional_probabilities returns out of the total selected-isotope particle events, what is the probability it is associated with the list of other isotopes 
def conditional_probabilities(data, isotope):
    obs = data[abs(data[isotope]) > 0.0]
    partners = (abs(obs) > 0.0).astype(np.float64).mean()
    return partners[abs(partners) > 0.0].sort_values(ascending=False)

#counts the total particles associated to an isotope
def conditional_particle(data, isotope):
    obs = data[abs(data[isotope]) > 0.0].count()
    return obs.sort_values(ascending=False)

#isotope_pure returns a data frame of the selected isotope particle event and its impurities
def isotope_pure(data, isotope):
    obs = data[data[isotope] > 0.0]
    others = obs.drop(columns=isotope)
    pure = others.sum(axis=1) == 0.0
    others = obs[pure]
    return others

#probability_pure returns out of the total selected-isotope particle events, what is the probability that it is not associated with any other isotope
def probability_pure(data, isotope):
    obs = data[data[isotope] > 0.0]
    others = obs.drop(columns=isotope)
    pure = (others.sum(axis=1) == 0.0).mean()
    return pure

#returns the same number of particles to the smallest dataframe
def sameamount(dfA, dfB):
    if len(dfA) > len(dfB):
        dfC = pd.concat([dfA[:len(dfB)], dfB], sort=False).fillna(0.0).clip(lower=0.0)
    else:
        dfC = pd.concat([dfA, dfB[:len(dfA)]], sort=False).fillna(0.0).clip(lower=0.0)

    labelsA = dfC.index.get_level_values(level='newlabel')
    return dfC, labelsA

def category_split(labeldf, label, upperbound, lowerbound):
        correct = labeldf[labeldf[label] > upperbound]
        uncertain = labeldf[labeldf[label] < upperbound]
        uncertain = uncertain[uncertain[label] > lowerbound]
        misclassified = labeldf[labeldf[label] < lowerbound]
        return [correct, uncertain, misclassified]
    
def graphmaker(df, label, upperbound, name, natdf, savefigs, opencircles = False):
    toppercent = conditional_probabilities(just_data(df[df[label] > upperbound]), '48Ti')
    bottompercent = conditional_probabilities(just_data(df[df[label] < upperbound]), '48Ti')
    percentdf = pd.DataFrame([toppercent, bottompercent]).T
    percentdf.columns = ['toppercent', 'bottompercent']
    percentdf.reset_index(inplace = True)
    percentdf = pd.melt(percentdf[1:13], id_vars = ['index'], value_vars = ['toppercent', 'bottompercent'])

    fig, axs = plt.subplots(1, 2, figsize=(16,4), dpi=300, gridspec_kw={'width_ratios': [2.5, 1]})
    ax1 = sns.barplot('index', 'value', 'variable', data = percentdf, ax = axs[0])
    ax1.set_ylabel('Isotope Frequency', fontsize = 16)
    ax1.tick_params(axis='both', which='major', labelsize=14)
    ax1.set_xlabel('')
    ax1.set_ylim([10**-3, 1])
    ax1.legend_.remove()
    ax1.set_yscale('log')
    
    #if you want unassociated Ti as a separate category
    if opencircles == True:
        #make the df pure ti df
        Pure = isotope_pure(just_data(df).drop(columns = '46Ti'), '48Ti')
        percentdfpure = df.loc[Pure.index]
        df.drop(percentdfpure.index, inplace = True)
        
    #parse particles by confidence probability to the correct category
    top = df[df[label] > upperbound]
    puretop = percentdfpure[percentdfpure[label] > upperbound]
    bot = df[df[label] < upperbound]
    purebot = percentdfpure[percentdfpure[label] < upperbound]

    #ax2 = sns.scatterplot('48Ti', label, data = percentdfpure, ax = axs[1], edgecolors = 'red')    
    axs[1].scatter(bot['48Ti'], bot[label], linewidths = 0.75, edgecolors = 'white', facecolors = '#ff7f0e')
    axs[1].scatter(top['48Ti'], top[label], linewidths = 0.75, edgecolors = 'white', facecolors = '#1f77b4')
    axs[1].scatter(purebot['48Ti'], purebot[label], linewidths = 0.75, edgecolors = 'white', facecolors = '#ff7f0e', marker = '^')
    axs[1].scatter(puretop['48Ti'], puretop[label], linewidths = 0.75, edgecolors = 'white', facecolors = '#1f77b4', marker = '^')
    axs[1].set_xscale('log')
    axs[1].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    axs[1].set_xlim([min(natdf['48Ti']), 10**-13])
    axs[1].set_xlabel('Ti Mass (g)')
    axs[1].set_ylabel('Prediction Probability of ' + str(label) + ' Category')
    axs[1].set_ylim(0.0)
    if savefigs == None:
        plt.savefig('../../figsfromscript/fingerprints' + str(label) + str(name) + '.png', dpi = 300)
    
def graphmakerm(df, label, upperbound, lowerbound, name, natdf, savefigs, opencircles = False):
    toppercent = conditional_probabilities(just_data(df[df[label] > upperbound]), '48Ti')
    middlepercent = df[df[label] < upperbound]
    middlepercent = conditional_probabilities(just_data(middlepercent[middlepercent[label] > lowerbound]), '48Ti')
    bottompercent = conditional_probabilities(just_data(df[df[label] < lowerbound]), '48Ti')
    percentdf = pd.DataFrame([toppercent, middlepercent, bottompercent]).T
    percentdf.columns = ['toppercent', 'middlepercent', 'bottompercent']
    percentdf.reset_index(inplace = True)
    percentdf = pd.melt(percentdf[1:13], id_vars = ['index'], value_vars = ['toppercent', 'middlepercent', 'bottompercent'])

    fig, axs = plt.subplots(1, 2, figsize=(18,4), dpi=300, gridspec_kw={'width_ratios': [3, 1]})
    ax1 = sns.barplot('index', 'value', 'variable', data = percentdf, palette = 'Blues', ax = axs[0])
    ax1.set_ylabel('Isotope Frequency')
    ax1.tick_params(axis='both', which='major', labelsize=12)
    ax1.set_xlabel('')
    ax1.legend_.remove()
    ax1.set_yscale('log')
    
    #if opencircles == True:
        #Pure = isotope_pure(just_data(df).drop(columns = '46Ti'), label)

    top = df[df[label] > upperbound]
    top['category'] = 'Top ' + str(100-upperbound*100) + '%'
    middle = df[df[label] < upperbound]
    middle = middle[middle[label] > lowerbound]
    middle['category'] = 'Middle' + str((upperbound - lowerbound)*100) + '%'
    bot = df[df[label] < lowerbound]
    bot['category'] = 'Bottom ' + str(upperbound*100) + '%'


    df = pd.concat([bot, middle, top], axis = 0)

    ax2 = sns.scatterplot('48Ti', label, 'category', data = df, palette = 'Blues_d', ax = axs[1])
    ax2 = sns.scatterplot('48Ti', label, 'category', data = percentdfpure, palette = 'Blues_d', ax = axs[1], markers= '+')
    ax2.set_xscale('log')
    ax2.set_xlim([min(natdf['48Ti']), 10**-13])
    ax2.set_xlabel('Ti Mass (g)')
    ax2.legend_.remove()
    ax2.set_ylabel('Prediction Probability of ' + str(label) + ' Category')
    if savefigs == None:
        plt.savefig('../../figsfromscript/fingerprints' + str(label) + str(name) + '.png', dpi = 300)
    
def get_ti(x):
    return x[:, [6]]


#isotope_notisotope returns a data frame of the selected isotope particle event not related to another selected isotope
def isotope_notisotope(data, isotope1, isotope2):
    obs = data[data[isotope1] > 0.0]
    obs = obs[obs[isotope2] == 0.0]
    return obs